In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
sample_submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train_set = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_set = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
y_train = train_set['SalePrice'].copy()

In [5]:
train_set

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [6]:
train_set.drop(['Alley', 'PoolQC', 'MiscFeature'], axis=1, inplace=True)

test_set.drop(['Alley', 'PoolQC', 'MiscFeature'], axis=1, inplace=True)

In [7]:
train_set.drop(['SalePrice'], axis=1, inplace=True)

In [8]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [9]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])
num_attribs = list(train_set.select_dtypes(exclude=['object']))
cat_attribs = list(train_set.select_dtypes(include=['object']))
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs)
])
housing_prepared = full_pipeline.fit_transform(train_set)

In [10]:
housing_prepared

<1460x293 sparse matrix of type '<class 'numpy.float64'>'
	with 112420 stored elements in Compressed Sparse Row format>

In [11]:
#Linear Regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, y_train)
some_data = train_set.iloc[:5]
some_labels = y_train.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
lin_reg.predict(some_data_prepared)

array([206587.53044441, 205688.56739047, 205073.06583248, 165128.73628504,
       291231.20891256])

In [12]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

20744.162237893146

In [13]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, y_train)
tree_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(y_train, housing_predictions)
tree_rsme = np.sqrt(tree_mse)
tree_rsme

20744.162237893146

In [14]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, y_train, scoring='neg_mean_squared_error', cv=3)
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([38410.55602121, 42416.22113714, 49896.28455236])

In [15]:
print("Mean: ", tree_rmse_scores.mean())
print("Standard deviation:", tree_rmse_scores.std())

Mean:  43574.35390356628
Standard deviation: 4760.00305331252


In [16]:
lin_scores = cross_val_score(lin_reg, housing_prepared, y_train, scoring='neg_mean_squared_error', cv=3)
lin_rmse_scores = np.sqrt(-lin_scores)
lin_rmse_scores

array([26189.09143006, 33571.61408934, 39835.61056508])

In [17]:
print("Mean: ", lin_rmse_scores.mean())
print("Standard deviation:", lin_rmse_scores.std())

Mean:  33198.772028161744
Standard deviation: 5577.40259305557


In [18]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, y_train)
forest_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(y_train, housing_predictions)
forest_rsme = np.sqrt(forest_mse)
forest_rsme

20744.162237893146

In [19]:
forest_scores = cross_val_score(forest_reg, housing_prepared, y_train, scoring='neg_mean_squared_error', cv=3)
forest_rmse_scores = np.sqrt(-forest_scores)
forest_rmse_scores

array([25969.61088469, 31877.69069806, 33348.73025966])

In [20]:
print("Mean: ", forest_rmse_scores.mean())
print("Standard deviation:", forest_rmse_scores.std())

Mean:  30398.677280803462
Standard deviation: 3188.882897798385


In [21]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]}
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring = 'neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [22]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30)

In [23]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

45949.437928147825 {'max_features': 2, 'n_estimators': 3}
39871.38391077049 {'max_features': 2, 'n_estimators': 10}
37513.69498446592 {'max_features': 2, 'n_estimators': 30}
45750.02070092519 {'max_features': 4, 'n_estimators': 3}
38559.607958976085 {'max_features': 4, 'n_estimators': 10}
36369.51191377811 {'max_features': 4, 'n_estimators': 30}
42721.92984457187 {'max_features': 6, 'n_estimators': 3}
36246.69602603405 {'max_features': 6, 'n_estimators': 10}
34657.49578288605 {'max_features': 6, 'n_estimators': 30}
41559.26777902299 {'max_features': 8, 'n_estimators': 3}
35210.71781821018 {'max_features': 8, 'n_estimators': 10}
33670.86677615714 {'max_features': 8, 'n_estimators': 30}
46467.11623199826 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
38113.42302301273 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
44433.95209675631 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
38029.94827770176 {'bootstrap': False, 'max_features': 3, 'n_estimators': 

In [24]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([5.56225907e-03, 6.26946689e-03, 1.31974311e-02, 3.18168145e-02,
       5.72794887e-02, 2.77399339e-03, 2.09215690e-02, 1.76818068e-02,
       2.04934189e-02, 2.79367641e-02, 1.30123570e-03, 8.22236193e-03,
       3.35446815e-02, 3.68515364e-02, 2.62119224e-02, 5.94027111e-04,
       5.49379288e-02, 4.27177427e-03, 3.70810437e-04, 1.36905887e-02,
       3.83317882e-03, 7.30042099e-03, 7.66023706e-04, 1.85324712e-02,
       1.43453635e-02, 2.24317766e-02, 2.56025400e-02, 2.18093235e-02,
       8.63654579e-03, 9.11573729e-03, 2.14732576e-03, 2.05814003e-04,
       1.99194978e-03, 1.11857599e-03, 8.26800803e-04, 5.67541058e-03,
       2.80787455e-03, 2.95547875e-04, 8.33956505e-04, 8.79623777e-05,
       2.55779140e-03, 2.58811210e-03, 4.01571813e-05, 6.54961255e-05,
       3.28410176e-03, 1.54296994e-03, 2.80514510e-04, 2.17034881e-03,
       5.02123590e-04, 9.34660857e-04, 4.37662519e-04, 2.16449835e-03,
       0.00000000e+00, 1.56508224e-06, 1.58312100e-03, 1.00413970e-03,
      

In [25]:
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)
# maybe drop less useful features

[(0.05727948871327229, 'OverallQual'),
 (0.0549379287590244, 'GrLivArea'),
 (0.036851536397949664, '1stFlrSF'),
 (0.033544681516570134, 'TotalBsmtSF'),
 (0.03181681448586311, 'LotArea'),
 (0.02793676413628692, 'BsmtFinSF1'),
 (0.026211922408442622, '2ndFlrSF'),
 (0.025602539965787992, 'GarageCars'),
 (0.022431776612095153, 'GarageYrBlt'),
 (0.021809323495545443, 'GarageArea'),
 (0.02092156899338216, 'YearBuilt'),
 (0.020493418940507085, 'MasVnrArea'),
 (0.01853247115998215, 'TotRmsAbvGrd'),
 (0.017681806752819575, 'YearRemodAdd'),
 (0.014345363466291627, 'Fireplaces'),
 (0.013690588699352468, 'FullBath'),
 (0.013197431076826789, 'LotFrontage'),
 (0.009115737287147223, 'OpenPorchSF'),
 (0.008636545786867227, 'WoodDeckSF'),
 (0.008222361927866276, 'BsmtUnfSF'),
 (0.007300420987478966, 'BedroomAbvGr'),
 (0.006269466887833161, 'MSSubClass'),
 (0.005675410575374169, 'MoSold'),
 (0.005562259072746986, 'Id'),
 (0.00427177427499193, 'BsmtFullBath'),
 (0.003833178819969211, 'HalfBath'),
 (0.002

In [26]:
final_model = grid_search.best_estimator_
X_test= test_set.copy()

In [27]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [28]:


X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_predictions

array([124948.16666667, 148251.66666667, 180342.9       , ...,
       157096.66666667, 116488.33333333, 221921.46666667])

In [29]:
test_ID = test_set['Id']
sub = pd.DataFrame({'Id':test_ID, 'SalePrice':final_predictions})
sub

,Id,SalePrice
0,1461,124948.166667
1,1462,148251.666667
2,1463,180342.900000
3,1464,194248.000000
4,1465,192073.333333
...,...,...
1454,2915,93923.333333
1455,2916,97706.666667
1456,2917,157096.666667
1457,2918,116488.333333


In [30]:
sub.to_csv('submission.csv', index=False)